In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import sys
import os

sys.path.append('../')
from functions.geometry import angle_between
from functions.tracking import update_landmarks_dict

In [2]:
# remove all files in data/tmp_frames  directory
for file in os.listdir('../data/tmp_frames'):
    os.remove(f'../data/tmp_frames/{file}')

In [21]:
create_output_video = False
local_minimum_threshold = 0.005
degree_threshold_parallel_pedals = 1

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# inputs
seconds_to_skip_beginning = 30
seconds_to_skip_end = 15
path_video = '../data/IMG_1476.mov'

# output
out_filename = 'tmp.mp4'

# variables
cap = cv2.VideoCapture(path_video)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
duration_seconds = n_frames/fps
frames_to_skip_beginning = int(round(seconds_to_skip_beginning * fps))
frames_to_skip_end = int(round(seconds_to_skip_end * fps))

# Define the codec and create VideoWriter object
if create_output_video:
    fourcc = cv2.VideoWriter_fourcc(*'H264') # for streamlit compatibility
    out = cv2.VideoWriter(out_filename, fourcc, fps, (width,  height))

I0000 00:00:1735735688.330270  541260 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-20.2.48), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine


W0000 00:00:1735735689.454481  551027 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735735689.627708  551027 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


#### Reding and preprocessing

In [4]:
frame_count = 0
pose_undetected = 0

landmarks_dict = {
    'left_hip': [],
    'left_knee': [],
    'left_ankle': [],
    'right_ankle': [],
    'left_shoulder': [],
    'left_elbow': [],
    'left_wrist': [],
}

frame_list = []

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# is there a way to process it faster and not frame by frame?
# possibility to save the edited video with landmarks and stuff
while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1

    if not ret:
        break
    
    if  frames_to_skip_beginning <= frame_count < (n_frames - frames_to_skip_end):

        # Convert the frame to RGB: why this conversion to writeable?
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Process the frame with MediaPipe Pose
        results = pose.process(image)
        
        # Convert the frame back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            update_landmarks_dict(results,width, height, landmarks_dict)
            frame_list.append(frame_count - frames_to_skip_beginning + 1)

            # Display angle close to the knee
            #cv2.rectangle(image, (int(knee[0]), int(knee[1]) - 20), (int(knee[0]) + 40, int(knee[1]) + 10), (0, 0, 0), -1)
            #cv2.putText(image, f'{int(angle)}', (int(knee[0]) + 5, int(knee[1]) + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (256, 256, 256), 1, cv2.LINE_AA)
        else:
            pose_undetected += 1

        if create_output_video:
            out.write(image)

        # save the image as compressed jpg 
        cv2.imwrite(f'../data/tmp_frames/frame_{frame_count - frames_to_skip_beginning + 1}.jpg', image)

cv2.destroyAllWindows()
cv2.waitKey(1)  # without this, will not close properly
cap.release()
if create_output_video:
    out.release()


W0000 00:00:1735734754.804179  541531 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735734755.454149  541531 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735734780.410487  541535 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [22]:
# create a dataframe with the data ankle, knee, hip
df_tracking = pd.DataFrame({
    'frame': frame_list,
    'hip': landmarks_dict['left_hip'],
    'knee': landmarks_dict['left_knee'],
    'ankle': landmarks_dict['left_ankle'],
    'ankle_right': landmarks_dict['right_ankle'],
    'shoulder': landmarks_dict['left_shoulder'],
    'elbow': landmarks_dict['left_elbow'],
    'wrist': landmarks_dict['left_wrist'],
})

# calculate the angle between the hip, knee and ankle
df_tracking['angle_knee'] = df_tracking.apply(lambda x: angle_between(x['knee'], x['hip'], x['ankle']), axis=1)
df_tracking['angle_shoulder'] = df_tracking.apply(lambda x: angle_between(x['shoulder'], x['hip'], x['elbow']), axis=1)
df_tracking['angle_elbow'] = df_tracking.apply(lambda x: angle_between(x['elbow'], x['shoulder'], x['wrist']), axis=1)
df_tracking['angle_torso'] = df_tracking.apply(lambda x: angle_between(x['hip'], x['shoulder'], x['hip'] - (1, 0)), axis=1)
df_tracking['angle_pedal'] = df_tracking.apply(lambda x: angle_between(x['ankle_right'], x['ankle'], x['ankle_right'] - (1, 0)), axis=1)

# boolean columns to identify positions
min_ankle = max([ankle[1] for ankle in df_tracking['ankle']])
df_tracking['min_height_ankle'] = df_tracking['ankle'].apply(lambda x: abs(x[1] - min_ankle)/min_ankle < local_minimum_threshold)
df_tracking['parallel_pedals'] = df_tracking['angle_pedal'].apply(lambda x: abs(x) < degree_threshold_parallel_pedals)

df_tracking.to_csv('../data/tmp_tracking.csv', index=False)

In [23]:
df_tracking[df_tracking['parallel_pedals'] == True] 

,frame,hip,knee,ankle,ankle_right,shoulder,elbow,wrist,angle_knee,angle_shoulder,angle_elbow,angle_torso,angle_pedal,min_height_ankle,parallel_pedals
5,6,"[232.74791479110718, 322.3212432861328]","[182.95008659362793, 373.0067443847656]","[232.26747751235962, 433.8457489013672]","[201.9395899772644, 433.83914947509766]","[173.70402932167053, 260.3915023803711]","[143.7402355670929, 305.14591217041016]","[100.56472778320312, 332.09144592285156]",96.477180,77.436396,155.770942,46.366546,0.012468,False,True
174,175,"[233.36802005767822, 320.93494415283203]","[183.97878885269165, 372.58716583251953]","[233.2995057106018, 434.2494583129883]","[188.7179946899414, 433.7233352661133]","[175.61059713363647, 257.6009178161621]","[144.02237176895142, 303.875732421875]","[102.0018982887268, 330.9659194946289]",97.628377,76.681551,157.127800,47.636770,0.676137,False,True
529,530,"[231.8258571624756, 321.3195037841797]","[182.84241199493408, 373.0521011352539]","[228.57512712478638, 434.68605041503906]","[191.36696577072144, 434.6043395996094]","[173.40667963027954, 259.2483139038086]","[141.92831754684448, 303.8423728942871]","[103.45016241073608, 331.06468200683594]",99.987952,78.481687,160.496196,46.736079,0.125824,False,True
578,579,"[232.31931924819946, 320.9868621826172]","[182.12875127792358, 369.6869659423828]","[226.45122528076172, 434.50172424316406]","[187.96774864196777, 434.1345977783203]","[174.88007068634033, 257.24634170532227]","[143.4865415096283, 302.4367904663086]","[102.47124195098877, 330.9309387207031]",99.770902,76.810809,159.575954,47.976664,0.546576,False,True
774,775,"[231.9705891609192, 321.9102478027344]","[185.08003950119019, 374.0456771850586]","[230.90669631958008, 437.92449951171875]","[191.32269859313965, 438.01116943359375]","[170.48474550247192, 265.4379653930664]","[148.03968787193298, 314.24816131591797]","[101.65368318557739, 333.08589935302734]",102.376169,72.128806,136.797511,42.566229,0.125450,False,True
868,869,"[231.81514978408813, 321.18812561035156]","[184.53834056854248, 371.6566467285156]","[229.82117414474487, 429.56939697265625]","[190.73557376861572, 428.91082763671875]","[166.40141487121582, 278.38085174560547]","[150.02583146095276, 326.20948791503906]","[101.78362011909485, 334.7731018066406]",98.847858,75.699136,118.966079,33.201058,0.965309,False,True
964,965,"[231.52596473693848, 321.1943817138672]","[182.78466939926147, 372.38624572753906]","[232.90616512298584, 432.0201110839844]","[199.87227201461792, 432.0256805419922]","[172.94787168502808, 260.7217788696289]","[142.91558504104614, 305.41133880615234]","[101.05380177497864, 330.78128814697266]",96.358111,77.990200,155.119471,45.911698,0.009660,False,True
1060,1061,"[231.8575930595398, 320.49705505371094]","[184.64614391326904, 371.3025665283203]","[224.3884563446045, 436.1842727661133]","[189.6040678024292, 436.56543731689453]","[174.31577682495117, 260.613956451416]","[144.51726078987122, 305.8839416503906]","[102.32205748558044, 331.28856658935547]",105.611042,77.212269,154.405555,46.142243,0.627817,False,True
1203,1204,"[231.42916917800903, 321.09203338623047]","[186.2079405784607, 373.86486053466797]","[230.07405281066895, 437.80086517333984]","[190.63339233398438, 437.56702423095703]","[172.84533619880676, 258.6774253845215]","[143.87730717658997, 303.1841468811035]","[101.60921216011047, 330.8860397338867]",104.952820,76.245479,156.299090,46.813362,0.339699,False,True
